In [13]:
import requests
from lxml import etree
import csv
import re
import folium
import pandas as pd
import time
import json

In [21]:
debug = False

In [23]:
def get_content_as_HTML(url):
    res = requests.get(url)
    raw = res.text
    html = etree.HTML(raw)
    if debug:
        print(raw)
    return html

In [24]:
def get_latlng(query):
    r=requests.get(f"https://www.google.com/maps/place/{query}")
    match = re.search("window.APP_INITIALIZATION_STATE=\[\[\[\d+\.\d+,(.*),(.*)\]",r.text)
    return float(match.group(2)), float(match.group(1))

In [25]:
def get_latlng2(query):
    api_key = ''
    r = requests.get(f'https://maps.googleapis.com/maps/api/geocode/json?address={query}&key={api_key}')
    resp = json.loads(r.text)
    coodinate = resp['results'][0]['geometry']['location']
    return coodinate['lat'], coodinate['lng']

## Get LA infected data

In [26]:
url = 'http://www.publichealth.lacounty.gov/media/Coronavirus/locations.htm'
html = get_content_as_HTML(url)

In [27]:
b = html.xpath('//*[@id="content"]/div/table/tbody/tr/th')
t = html.xpath('//*[@id="content"]/div/table/tbody/tr/td')

In [28]:
csv_file = open('result.csv', 'w', newline='')
writer = csv.writer(csv_file)
writer.writerow(['city', 'case_num'])
flag = False
for i,j in zip(b,t):
    if flag ==False:
        flag = i.text.startswith('CITY')
    else:
        if i.text.startswith('Under Investigation'):
            continue
        writer.writerow([i.text, j.text])
csv_file.close()

if debug:
    with open('result.csv','r') as csvFile:
        reader = csv.reader(csvFile)
        for line in reader:
                  print(line)

In [29]:
la_data = pd.read_csv('result.csv')
la_data.info()
la_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   city      109 non-null    object
 1   case_num  109 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 1.8+ KB


,city,case_num
0,Agoura Hills,1
1,Alhambra,4
2,Altadena,5
3,Arcadia,3
4,Athens,1


## Get coordinates from google

In [30]:
coordinates = []
for _,row in la_data.iterrows():
    cood = get_latlng2(row['city']+', Los Angeles, CA')
    print(row['city'], cood)
    coordinates.append(cood)
    time.sleep(0.3)
print('------finished query----')

Agoura Hills (34.1533395, -118.7616764)
Alhambra (34.095287, -118.1270146)
Altadena (34.190162, -118.1313188)
Arcadia (34.1397292, -118.0353449)
Athens (33.9200153, -118.2811844)
Baldwin Hills (34.006677, -118.3505779)
Bellflower (33.8816818, -118.1170117)
Beverly Hills (34.0736204, -118.4003563)
Beverlywood (34.0494132, -118.3952319)
Boyle Heights (34.0297895, -118.2117257)
Brentwood (34.0521011, -118.4732464)
Burbank (34.1808392, -118.3089661)
Calabasas (34.1367208, -118.6614809)
Canoga Park (34.208254, -118.6058609)
Carson (33.8316745, -118.281693)
Castaic (34.4890363, -118.6256552)
Century City (34.0536608, -118.4133894)
Century Palms/Cove (33.9453293, -118.2732046)
Cerritos (33.8583483, -118.0647871)
Cloverdale/Cochran (34.0312771, -118.3578858)
Compton (33.8958492, -118.2200712)
Covina (34.0900091, -117.8903397)
Crestview (34.0478848, -118.3804254)
Culver City (34.0211224, -118.3964665)
Diamond Bar (34.0286226, -117.8103367)
Downtown (34.040713, -118.2467693)
Duarte (34.1394513, 

In [31]:
geo_data=pd.DataFrame({"city":la_data["city"],"lat":list(zip(*coordinates))[0],"lng":list(zip(*coordinates))[1]})
geo_data = geo_data[geo_data['city']!="Under Investigation"]
geo_data

,city,lat,lng
0,Agoura Hills,34.153340,-118.761676
1,Alhambra,34.095287,-118.127015
2,Altadena,34.190162,-118.131319
3,Arcadia,34.139729,-118.035345
4,Athens,33.920015,-118.281184
...,...,...,...
104,Whittier,33.979179,-118.032844
105,Wilmington,33.785795,-118.264357
106,Wilshire Center,34.065381,-118.300369
107,Winnetka,34.204859,-118.573962


In [32]:
latlng_data = pd.DataFrame({"city":la_data["city"],"lat":list(zip(*coordinates))[0],"lng":list(zip(*coordinates))[1],"infected":la_data["case_num"]})

In [33]:
latlng_data
t = latlng_data[latlng_data['city']!="Under Investigation"]
t[t['city']=='Park LaBrea']

,city,lat,lng,infected
63,Park LaBrea,34.070047,-118.355219,6


In [34]:
m = folium.Map(location=[latlng_data['lat'].mean(), latlng_data['lng'].mean()])
for _,row in t.iterrows():
    folium.Circle(
        location=[row['lat'], row['lng']],
        popup = row['city']+f' -> {row["infected"]}',
        radius = 100+row['infected']*100,
        color='crimson',
        fill=True,
        fill_color="crimson"
        ).add_to(m)

In [35]:
m